# XCS in Multiplexer

In [1]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

import gym
import gym_multiplexer  

from lcs.agents import EnvironmentAdapter
from lcs.agents.xcs import XCS, Configuration

# Environment

In [2]:
mpx = gym.make('boolean-multiplexer-6bit-v0')

Example of return value

In [3]:
mpx.reset()

[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]

The last bit is changed after executing an action. It if was valid it's set to `1`, otherwise `0`. This behavior is not needed for the XCS, therefore signal can becompacted

## Agent

In [20]:
class MultiplexerAdapter(EnvironmentAdapter):
    @classmethod
    def to_genotype(cls, env_state):
        return list(map(str, map(int, env_state[:-1])))  # skip last bit

def mpx_metrics(xcs: XCS, environment):
    return {
        'population': len(xcs.population),
        'numerosity': sum(cl.numerosity for cl in xcs.population)
    }
    
cfg = Configuration(number_of_actions=2,
                    max_population=1000,
                    gamma=0.9,
                    metrics_trial_frequency=100,
                    environment_adapter=MultiplexerAdapter(),
                    user_metrics_collector_fcn=mpx_metrics)

agent = XCS(cfg)

## Experiments

In [24]:
%%time
explore_population, explore_metrics = agent.explore(mpx, 10_000, decay=False)

INFO:lcs.agents.Agent:{'trial': 0, 'steps_in_trial': 1, 'reward': [570.5508815523286, 530.0908629315612], 'perf_time': 0.0009067079954547808, 'population': 15, 'numerosity': 372}
INFO:lcs.agents.Agent:{'trial': 1000, 'steps_in_trial': 1, 'reward': [517.6587436649028, 369.43516701389626], 'perf_time': 0.00020945801225025207, 'population': 22, 'numerosity': 833}
INFO:lcs.agents.Agent:{'trial': 2000, 'steps_in_trial': 1, 'reward': [554.7412290238851, 364.03865588843746], 'perf_time': 0.0002216589928139001, 'population': 28, 'numerosity': 1000}
INFO:lcs.agents.Agent:{'trial': 3000, 'steps_in_trial': 1, 'reward': [563.2684490900601, 580.2952947713835], 'perf_time': 0.00021999901218805462, 'population': 31, 'numerosity': 1000}
INFO:lcs.agents.Agent:{'trial': 4000, 'steps_in_trial': 1, 'reward': [679.0461013929137, 443.2434086252577], 'perf_time': 0.00022068100224714726, 'population': 32, 'numerosity': 1000}
INFO:lcs.agents.Agent:{'trial': 5000, 'steps_in_trial': 1, 'reward': [652.42012436553

CPU times: user 3.06 s, sys: 0 ns, total: 3.06 s
Wall time: 3.05 s


In [26]:
print(f'Explore population count: {len(explore_population)}\n')

print('Top 15 classifiers:')
for cl in sorted(explore_population, key=lambda cl: -cl.fitness)[15:]:
    print(f'{cl.condition}\t{cl.action}')

Explore population count: 33

Top 15 classifiers:
###00#	0
0#01##	0
#00#10	1
1#####	0
01#00#	1
000##1	1
#1##1#	0
#1####	0
#1##1#	1
10#11#	0
#1##1#	0
11####	1
#11101	1
#1##1#	1
111###	1
###001	1
####0#	0
####0#	1


See if there are some duplicate classifiers.

In [40]:
rules = [(cl.condition, cl.action) for cl in explore_population]
len(set(rules))

31

There are duplicate classifiers found.